# Levelling up your Julia Skills
by Brian Jackson

## Environment Files

`Project.toml` file
- records all the project dependencies
- easily add dependencies using the package manager by activating the project file via `] activate <path/to/project>`
- Julia will first search in the local project dependencies, and then resort to globally install projects (i.e. you can still use projects installed in your default global environment)

`Manifest.toml` file
- records the specific versions of all projects you are using (including locally-referenced projects)
- essentially captures a "snapshot" of your work environement
- automatically generated/updated by `] resolve`
- do NOT edit manually
- should NOT be committed to packages
- can be commited for examples and testing environements

Some suggestions
- To avoid adding lots of dependencies to your project, set up `Project.toml` and `Manifest.toml` files in an `examples/` sub-directory that record dependencies needed for plotting, analysis, setting up test problems, etc.
- If you need to "ship" some code to someone else and want to make sure it works, commit the `Manifest.toml` file. The recipient then clones the repo, activates the repo, and then downloads all dependencies using `] instantiate`
- Read the docs https://julialang.github.io/Pkg.jl/v1/
- Check out this talk from JuliaCon 2019 https://www.youtube.com/watch?v=q-LV4zoxc-E

## Constructors

### Inner Constructors
* must call `new`
* use to establish representational invariants
* `new` will automatically call `convert` on the arguments

In [65]:
using StaticArrays
using BenchmarkTools

struct UnitQuaternion{T}
    w::T
    x::T
    y::T
    z::T

    @inline function UnitQuaternion{T}(w, x, y, z, normalize::Bool = true) where T
        if normalize
            inorm = inv(sqrt(w*w + x*x + y*y + z*z))
            new{T}(w*inorm, x*inorm, y*inorm, z*inorm)
        else
            new{T}(w, x, y, z)
        end
    end

    UnitQuaternion{T}(q::UnitQuaternion) where T = new{T}(q.w, q.x, q.y, q.z)
end

In [66]:
# Element-wise constructor
q1 = UnitQuaternion{Float64}(1.0, 2.0, 3.0, 4.0)        # All Float64, no conversion needed
println(typeof(q1))
println(typeof(UnitQuaternion{Float64}(1, 0, 0, 0)))    # convert inputs to Float64
println(typeof(UnitQuaternion{Int}(1, 0, 0, 0)))        # All Ints, no conversion needed
# println(typeof(UnitQuaternion{Int}(1.0, 0, 0, 0)))    # error, since a Float64 cannot be converted to Int64
println(typeof(UnitQuaternion{Float32}(1.0, 0, 0, 0)))  # convert all inputs to Float32

# Copy constructor
println(typeof(UnitQuaternion{Float32}(q1)))             # convert Float64 quaternion to Float32
println(typeof(UnitQuaternion{Float64}(q1)))             # no conversion needed

UnitQuaternion{Float64}
UnitQuaternion{Float64}
UnitQuaternion{Int64}
UnitQuaternion{Float32}
UnitQuaternion{Float32}
UnitQuaternion{Float64}


In [40]:
# This method isn't defined yet
UnitQuaternion(1.0, 0.0, 0.0, 0.0)

UnitQuaternion{Float64}(1.0, 0.0, 0.0, 0.0)

In [67]:
# Use promote to handle different input types
function UnitQuaternion(w,x,y,z, normalize::Bool = true)
    types = promote(w,x,y,z)
    UnitQuaternion{eltype(types)}(w,x,y,z, normalize)
end
println(typeof(UnitQuaternion(1.0, 0.0, 0.0, 0.0)))     # detects Float64
println(typeof(UnitQuaternion(1.0, 0, 0, 0)))           # promotes to Float64
println(typeof(UnitQuaternion(1.0f0, 0, 0, 0)))         # promotes to Float32

UnitQuaternion{Float64}
UnitQuaternion{Float64}
UnitQuaternion{Float32}


In [8]:
# Pass in vectors
@inline function (::Type{Q})(q::AbstractVector, normalize::Bool = true) where Q <: UnitQuaternion
    Q(q[1], q[2], q[3], q[4], normalize)
end
println(UnitQuaternion([1.0, 0, 0, 0]))           # passes through promotion constructor
println(UnitQuaternion{Float32}([1.0, 0, 0, 0]))  # call inner constructor (uses internal call to convert)

UnitQuaternion{Float64}(1.0, 0.0, 0.0, 0.0)
UnitQuaternion{Float32}(1.0f0, 0.0f0, 0.0f0, 0.0f0)


In [9]:
# Pass-through copy constructor 2
UnitQuaternion(q::UnitQuaternion) = q

UnitQuaternion

## Using Static Arrays

`StaticArray{S,T,N} <: AbstractArray{T,N}`
- abstract supertype that is used to define common methods
- simply an abstract array with a size tuple `S`


`SArray{S,T,N,L}`
- static matrix that lives on the stack, rather than the heap (`isbits(a::SArray) = true`)
- underlying storage is a tuple of length `L`
- very efficient to create small-sized arrays
- don't use for dimensions above 20x20
- very slow initialization and compilation for sizes above 20x20

`MArray{S,T,N,L}`
- similar to `SArray` but has mutuable elements so lives on the heap


`SizedArray{S,T,N,M}`
- simply a normal Julia array of dimension `M` wrapped with a static size
- fast to initialize using `StaticMatrix{n,m}(zeros(n,m))` for any size
- for small matrix multiplication (less than 14x14x14) uses loop unrolling
- for large arrays, uses BLAS
- best of both, but for 0 allocations you have to use in-place methods (e.g. `mul!`)
- can be converted very quickly to `SArray` at run-time (as long as the sizes are small enough)

### Some Practical Tips
- When pre-allocating memory that could potentially have dimensions bigger than 20x20, used `SizedArray`
- You can get extra speed for small problems by converting them to `SMatrix` temporarily (and easier-to-read math)
- Use `mul!(C,A,B)` and `mul!(C,A,B,alpha,beta)` for matrix multiplication
- Never EVER store static arrays of different sizes in the same vector
- If your arrays aren't too large, expressions like the following don't allocate: `C .= A*B .+ D`

In [68]:
C = @MMatrix zeros(4,5)
A = @MMatrix rand(4,6)
B = @MMatrix rand(6,5)
D = @MMatrix rand(4,5)
@btime $C .= $A*$B .+ $D

  31.260 ns (0 allocations: 0 bytes)


4×5 MArray{Tuple{4,5},Float64,2,20} with indices SOneTo(4)×SOneTo(5):
 1.25518  1.22202  1.83152  0.706902  1.95337
 1.82795  1.60956  1.71565  1.69904   1.78458
 2.12251  1.52831  2.53513  1.40619   1.7827
 2.87063  1.96925  2.55051  1.93487   2.61875

## Function Barriers
An efficient way to eliminate allocations for general code

### Sample problem:
Find the maximum constraint violation. We have 3 constraints of lengths 4, 6, and 8. Each constraint is applied to 10 time steps.

In [69]:
data = [[@SVector rand(n) for k = 1:10] for n in (4,6,8)];
typeof(data)  # potential type instability

Array{Array{T,1} where T,1}

In [70]:
function findmax(data)
    mymax = -Inf
    for (i,vec) in enumerate(data)
        for x in vec
            mymax = max(maximum(x), mymax)
        end
    end
    return mymax
end
@btime findmax($data)

  1.560 μs (90 allocations: 2.81 KiB)


0.9981549078964251

In [71]:
@code_warntype findmax(data)

Variables
  #self#::Core.Compiler.Const(findmax, false)
  data::Array{Array{T,1} where T,1}
  mymax::Any
  @_4::Union{Nothing, Tuple{Tuple{Int64,Array{_A,1} where _A},Tuple{Int64,Int64}}}
  i::Int64
  vec::Array{_A,1} where _A
  @_7::Int64
  @_8::Union{Nothing, Tuple{Any,Int64}}
  x::Any

Body::Any
1 ─       (mymax = -Main.Inf)
│   %2  = Main.enumerate(data)::Base.Iterators.Enumerate{Array{Array{T,1} where T,1}}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #7 if not %5
2 ┄ %7  = @_4::Tuple{Tuple{Int64,Array{_A,1} where _A},Tuple{Int64,Int64}}::Tuple{Tuple{Int64,Array{_A,1} where _A},Tuple{Int64,Int64}}
│   %8  = Core.getfield(%7, 1)::Tuple{Int64,Array{_A,1} where _A}
│   %9  = Base.indexed_iterate(%8, 1)::Core.Compiler.PartialStruct(Tuple{Int64,Int64}, Any[Int64, Core.Compiler.Const(2, false)])
│         (i = Core.getfield(%9, 1))
│         (@_7 = Core.getfield(%9, 2))
│   %12 = Base.indexed_iterate(%8, 2, @_7::Cor

In [72]:
function findmax(data)
    mymax = -Inf
    for (i,vec) in enumerate(data)
        # function barrier: pass the type instability directly to a function
        tmp = findmax(vec)
        mymax = max(tmp, mymax)
    end
    return mymax
end

function findmax(vec::Vector{<:StaticVector{<:Any,T}}) where T
    mymax = typemin(T)
    for x in vec
        mymax = max(mymax,maximum(x))
    end
    return mymax
end
@btime findmax($data)

  283.219 ns (6 allocations: 96 bytes)


0.9981549078964251

In [73]:
function findmax!(maxes, data)
    mymax = -Inf
    for (i,vec) in enumerate(data)
        tmp = maxes[i]
        findmax!(tmp, vec)
        mymax = max(mymax,maximum(tmp))
    end
    return mymax
end
function findmax!(maxes, vec::Vector{<:StaticVector{<:Any,T}}) where T
    for (i,x) in enumerate(vec)
        maxes[i] = maximum(x)
    end
end
maxes = [zeros(10) for N in (4,6,8)]
@btime findmax!($maxes, $data)

  257.540 ns (0 allocations: 0 bytes)


0.9981549078964251

## Metaprogramming

What is metaprogramming?
- Using code to generate more code

Why use metaprogamming?
- Move computation from runtime to compile time => faster runtime code!
- Avoid unnecessary repetition by writing general functions

Why to not use metaprogramming?
- If you abuse it, it can have very poor performance, make your code completely unintelligble, and extremely hard to debug

When to use metaprogramming?
- As a last resort
- When you have lots of information from your types
- Elimination of potential type instability (e.g. your output is a `StaticArray`)

### Example: Matrix-Vector multiplication with Loop Unrolling

In [74]:
# Set up
sa = Size(3,4)  # size of A
sb = Size(4)    # Size of b

A = rand(sa[1], sa[2]);
b = rand(sb[1]);

In [75]:
# Step 1: Creating expressions
i,j = 2,1
:(A[$i,$j])

:(A[2, 1])

In [77]:
# Create expressions for multiplying first row of A with b
firstrow = [:(A[1,$j] * b[$j]) for j = 1:sb[1]]

4-element Array{Expr,1}:
 :(A[1, 1] * b[1])
 :(A[1, 2] * b[2])
 :(A[1, 3] * b[3])
 :(A[1, 4] * b[4])

In [79]:
# We can use linear indices instead of cartesian for even more speed
firstrow = [:(A[$(LinearIndices(sa)[1,j])] * b[$j]) for j = 1:sb[1]]

4-element Array{Expr,1}:
 :(A[1] * b[1])
 :(A[4] * b[2])
 :(A[7] * b[3])
 :(A[10] * b[4])

In [81]:
# Reduce the expressions using addition
reduce((ex1,ex2) -> :($ex1 + $ex2), firstrow)

:(((A[1] * b[1] + A[4] * b[2]) + A[7] * b[3]) + A[10] * b[4])

In [83]:
# Do this for each row
exprs = [reduce((ex1,ex2) -> :($ex1 + $ex2), [:(A[$(LinearIndices(sa)[i,j])] * b[$j]) for j = 1:sb[1]]) for i = 1:sa[1]]

3-element Array{Expr,1}:
 :(((A[1] * b[1] + A[4] * b[2]) + A[7] * b[3]) + A[10] * b[4])
 :(((A[2] * b[1] + A[5] * b[2]) + A[8] * b[3]) + A[11] * b[4])
 :(((A[3] * b[1] + A[6] * b[2]) + A[9] * b[3]) + A[12] * b[4])

In [84]:
# Create the output vector
output = quote 
    SVector{$(sa[1])}($(exprs...))
end

quote
    #= In[84]:3 =#
    SVector{3}(((A[1] * b[1] + A[4] * b[2]) + A[7] * b[3]) + A[10] * b[4], ((A[2] * b[1] + A[5] * b[2]) + A[8] * b[3]) + A[11] * b[4], ((A[3] * b[1] + A[6] * b[2]) + A[9] * b[3]) + A[12] * b[4])
end

In [86]:
eval(output)  # never do this at run-time!

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 0.4812650430532941
 0.6383943890345716
 0.5104157415147225

In [88]:
# Now create a function that returns the expression we created above
function matvec_mult(::Size{sa}, ::Size{sb}, A, b) where {sa, sb}
    # Everything here must be a function of ONLY THE TYPES
    exprs = [reduce((ex1,ex2) -> :($ex1 + $ex2), [:(A[$(LinearIndices(sa)[i,j])] * b[$j]) for j = 1:sb[1]]) for i = 1:sa[1]]
    
    # Return an expression
    return quote
        SVector{$(sa[1])}($(exprs...))
    end
end
matvec_mult(sa, sb, A, b)

quote
    #= In[88]:8 =#
    SVector{3}(((A[1] * b[1] + A[4] * b[2]) + A[7] * b[3]) + A[10] * b[4], ((A[2] * b[1] + A[5] * b[2]) + A[8] * b[3]) + A[11] * b[4], ((A[3] * b[1] + A[6] * b[2]) + A[9] * b[3]) + A[12] * b[4])
end

In [90]:
# Now that we're sure the expression is what we want, stick @generated at the front
@generated function matvec_mult(::Size{sa}, ::Size{sb}, A, b) where {sa, sb}
    # Everything here must be a function of ONLY THE TYPES
    exprs = [reduce((ex1,ex2) -> :($ex1 + $ex2), [:(A[$i,$j] * b[$j]) for j = 1:sb[1]]) for i = 1:sa[1]]
    
    # Return an expression
    return quote
        SVector{$(sa[1])}($(exprs...))
    end
end
matvec_mult(sa, sb, A, b)

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 0.4812650430532941
 0.6383943890345716
 0.5104157415147225

In [91]:
@btime matvec_mult($sa, $sb, $A, $b)
@btime $A*$b;

  7.893 ns (0 allocations: 0 bytes)
  51.205 ns (1 allocation: 112 bytes)
